In [24]:
import requests, json, regex
from bs4 import BeautifulSoup

# Get Clinic Names
cities = ['Los Angeles','San Francisco','Phoenix-az','Las Vegas','paradise-nv','Denver','Seattle',
         'Portland','baltimore-md','silver springs-md','rockville-md']
count = 0
#initialize_db(0,init = True)
# Iterate over all cities and get data
for city in cities:
    base = 'https://www.leafly.com/finder/%s'
    city_id = initialize_db(city)
    print(city_id)
    city_url = base % city
    #get_city_clinic_data(city_url,city_id)
    print(city_url)
get_lat_lng()

Denver
5
total num clinics without failures 100
The Joint - Denver inserted already
Lightshade - Sheridan inserted already
L'Eagle Services LLC inserted already
Lightshade - Havana inserted already
Lightshade - Iliff inserted already
Lightshade - Peoria inserted already
Lightshade - Holly inserted already
Lightshade - 6th Avenue inserted already
Ballpark Holistic Dispensary inserted already
Altitude East Denver inserted already
Lightshade - Dayton inserted already
Lightshade - Federal Heights inserted already
Kind Love inserted already
Sacred Seed - Recreational 21+ inserted already
Botanico inserted already
Affinity Dispensary inserted already
Trenchtown inserted already
Diego Pellicer - Denver inserted already
Good Meds Englewood inserted already
Good Meds Lakewood inserted already
Good Chemistry inserted already
Life Flower Dispensary inserted already
Natural Selections inserted already
The Lodge Cannabis - High St. inserted already
Rocky Road Remedies Aurora inserted already
Good C

exception  Pot Shop Seattle 	 	 0
Pot Shop Seattle  	is null deleting...
https://www.leafly.com/finder/Seattle
Portland
7
total num clinics without failures 99
Brothers Cannabis - SE Morrison inserted already
MediGreen Collectives inserted already
Bridge City Collective - Southeast Portland & Delivery inserted already
Nectar - Barbur inserted already
Nectar - Sandy inserted already
Growing ReLeaf - Beaverton inserted already
Nectar - Mississippi inserted already
Puddletown Organics inserted already
Brothers Cannabis inserted already
Green Front inserted already
Green Mart - Beaverton, OR inserted already
Hi Cascade - Portland inserted already
AmericannaRx inserted already
Urban Farmacy inserted already
Green Goddess Remedies inserted already
Kaleafa Cannabis Co. - Portland inserted already
Nectar - Burlingame inserted already
Chalice Farms Powell inserted already
The Green Remedy inserted already
Nectar - Lents inserted already
Five Zero Trees Division inserted already
Bloom - Portland

exception  ReLeaf Shop 	 	 5
exception  Storehouse 	 	 3
exception  Pure Life Wellness MD 	 	 3
exception  Blair Wellness Center 	 	 3
this is unit [] from clinic name 	 398
this is unit [] from clinic name 	 398
this is unit [] from clinic name 	 398
this is unit [] from clinic name 	 398
this is unit [] from clinic name 	 398
this is unit [] from clinic name 	 398
this is unit [] from clinic name 	 398
this is unit [] from clinic name 	 398
this is unit [] from clinic name 	 398
this is unit [] from clinic name 	 398
this is unit [] from clinic name 	 398
this is unit [] from clinic name 	 398
Starbuds Baltimore good    8
exception  Culta LLC 	 	 0
Culta LLC  	is null deleting...
exception  Maggies MD 	 	 0
Maggies MD  	is null deleting...
exception  Mission - Baltimore (Coming Soon) 	 	 0
Mission - Baltimore (Coming Soon)  	is null deleting...
exception  Amedicanna Dispensary 	 	 0
Amedicanna Dispensary  	is null deleting...
https://www.leafly.com/finder/baltimore-md
silver springs-

In [16]:
# request into city url and get the list of clinics available
# get clinic names and generate url's
# function call to open_clinic_menu to request into clinic's menu page

# scrape for clinic names and menu urls
def get_city_clinic_data(city_url,city_id):
    import sqlite3
    # request into city leafly page
    r = requests.get(city_url)
    main_soup = BeautifulSoup(r.content,'lxml')    
    
    # Find clinic name's and associate menu url
    div_names = main_soup.select('div[class*="name-placement"]')
    div_urls = main_soup.select('div[class*="view-menu"]')        
    print('total num clinics without failures',len(div_names))
    
    for name,urls in zip(div_names,div_urls):        
        # Generate URL's
        url = urls.find('a').get('href')   
        ind =url.rfind('/')
        url = url[0:ind]
        base = 'https://www.leafly.com%s'
        clinic_url = base % url
        # Open Clinic URL/Main menu page
        get_clinic_menu(clinic_url,city_id,name.text)

        

In [17]:
# scrape clinic menu
def get_clinic_menu(clinic_url,city_id,clinic_name):
    import time, sqlite3
    # go to clinic page
    time.sleep(2)
    r = requests.get(clinic_url)
    soup = BeautifulSoup(r.content,'lxml')
    
    # check for duplicate clinics
    conn = sqlite3.connect('clinic_data.db')
    cur = conn.cursor()
    cur.execute('SELECT * FROM clinics WHERE name=?',(clinic_name,))
    check = cur.fetchone()
    if check is not None:
        print(clinic_name,'inserted already')
        cur.close()
        return
    
    # get address
    clinic_address = soup.select('div[class*="location"]')[1].text
    if 'Select' in clinic_address:   # skip clinics with no adress
        cur.execute('DELETE FROM clinics WHERE name=?',(clinic_name,))
        print('no real address, \t delete...!',clinic_name)
        conn.commit()
        cur.close()
        return 
    # get number of pages in menu
    try:
        num_pages = soup.select('span[class*="last-page-display"]')[0].text
        print(clinic_name,'good   ', num_pages)
    except IndexError:
        num_pages = soup.select('span[class*="paging"]')
        num_pages = len(num_pages)
        print('exception ',clinic_name, '\t \t' , num_pages) 

    if int(num_pages) == 0:
        print(clinic_name,' \tis null deleting...')
        cur.execute('DELETE FROM clinics WHERE name=?',(clinic_name,))
        conn.commit()
        cur.close()
        return 
    
    # store clinic meta data in db
    conn = sqlite3.connect('clinic_data.db')
    cur = conn.cursor()
    cur.execute('INSERT INTO clinics (name,address,city_id) VALUES (?,?,?)'
               ,(clinic_name,clinic_address,city_id))
    cur.execute('SELECT id FROM clinics WHERE name=?',(clinic_name,))
    clinic_id = cur.fetchone()[0]
    conn.commit()
    cur.close()
    
    # Get leafly dispensary id
    ind = clinic_url.rfind('/')
    ind2 = clinic_url.rfind('info')
    leafly_id = clinic_url[ind2+5:ind]

    # iterate over all the menu's pages
    for page in range(1,int(num_pages)+1):
        if page == 1:
            menu = soup.select('table[class*="menu-item"]') 
            get_menu_items(menu,clinic_id, city_id)           
        else:
            time.sleep(5)
            # Generate url to scrape all pages
            base = 'https://www.leafly.com/dispensary-info/%s/?menu-page=%s'
            menu_page = base % (leafly_id, str(page) )
            r = requests.get(menu_page)
            soup = BeautifulSoup(r.content,'lxml')
        
            # Get list of all menu items on page
            menu = soup.select('table[class*="menu-item"]') 
            get_menu_items(menu, clinic_id, city_id)

In [18]:
# scrape page menu
def get_menu_items(menu_soup,clinic_id, city_id):
    import sqlite3
    
    # iterate over menu items
    for item in menu_soup: 
        # get stran name
        name = item.select('span[class*="item-name"]')[0].text
        # get item price. only selecting items with 1/8 quantities 
        unit = item.select('div[class*="price"]')
        try:
            if '⅛' in unit[0].text:
                conn = sqlite3.connect('clinic_data.db')
                cur = conn.cursor()
            
                price = unit[1].text
                if city_id == 1:
                    cur.execute('INSERT INTO clinic_menus_LA (item_name,price,clinic_id) VALUES (?,?,?)',
                       (name,price,clinic_id))
                if city_id == 2:
                    cur.execute('INSERT INTO clinic_menus_SF (item_name,price,clinic_id) VALUES (?,?,?)',
                       (name,price,clinic_id))
                if city_id == 3:
                    cur.execute('INSERT INTO clinic_menus_AZ (item_name,price,clinic_id) VALUES (?,?,?)',
                       (name,price,clinic_id))
                if city_id == 4:
                    cur.execute('INSERT INTO clinic_menus_LV (item_name,price,clinic_id) VALUES (?,?,?)',
                       (name,price,clinic_id))
                if city_id == 5:
                    cur.execute('INSERT INTO clinic_menus_DEN (item_name,price,clinic_id) VALUES (?,?,?)',
                       (name,price,clinic_id))
                if city_id == 6:
                    cur.execute('INSERT INTO clinic_menus_SEA (item_name,price,clinic_id) VALUES (?,?,?)',
                       (name,price,clinic_id))
                if city_id == 7:
                    cur.execute('INSERT INTO clinic_menus_POR (item_name,price,clinic_id) VALUES (?,?,?)',
                       (name,price,clinic_id))
                if city_id == 8:
                    cur.execute('INSERT INTO clinic_menus_DET (item_name,price,clinic_id) VALUES (?,?,?)',
                       (name,price,clinic_id))
                if city_id == 9:
                    cur.execute('INSERT INTO clinic_menus_BAL (item_name,price,clinic_id) VALUES (?,?,?)',
                       (name,price,clinic_id))
                conn.commit()
                cur.close()
        except:
            print('this is unit',unit, 'from clinic name \t',clinic_id)
            continue       

https://www.leafly.com/dispensary-info/lak-


43 34
lak-


In [1]:
def initialize_db(city_name,**init):
    import sqlite3
    conn = sqlite3.connect('clinic_data.db')
    cur = conn.cursor()
    if 'init' in init and init['init'] == True:
        cur.execute('DROP TABLE IF EXISTS city_table')
        cur.execute('CREATE TABLE city_table (name TEXT, id INTEGER UNIQUE PRIMARY KEY NOT NULL)')
        
        cur.execute('DROP TABLE IF EXISTS clinics')
        cur.execute('''CREATE TABLE clinics (id INTEGER UNIQUE PRIMARY KEY NOT NULL, name TEXT,  
                    address TEXT,city_id INTEGER)''')
        
        cur.execute('DROP TABLE IF EXISTS clinic_menus_LA')
        cur.execute('''CREATE TABLE clinic_menus_LA (item_name TEXT, price INTEGER, 
                    clinic_id INTEGER,city_id INTEGER)''')        
        cur.execute('DROP TABLE IF EXISTS clinic_menus_SF')
        cur.execute('''CREATE TABLE clinic_menus_SF (item_name TEXT, price INTEGER, 
                    clinic_id INTEGER,city_id INTEGER)''')
        cur.execute('DROP TABLE IF EXISTS clinic_menus_AZ')
        cur.execute('''CREATE TABLE clinic_menus_AZ (item_name TEXT, price INTEGER, 
                    clinic_id INTEGER,city_id INTEGER)''')
        cur.execute('DROP TABLE IF EXISTS clinic_menus_LV')
        cur.execute('''CREATE TABLE clinic_menus_LV (item_name TEXT, price INTEGER, 
                    clinic_id INTEGER,city_id INTEGER)''')
        cur.execute('DROP TABLE IF EXISTS clinic_menus_DEN')
        cur.execute('''CREATE TABLE clinic_menus_DEN (item_name TEXT, price INTEGER, 
                    clinic_id INTEGER,city_id INTEGER)''')
        cur.execute('DROP TABLE IF EXISTS clinic_menus_SEA')
        cur.execute('''CREATE TABLE clinic_menus_SEA (item_name TEXT, price INTEGER, 
                    clinic_id INTEGER,city_id INTEGER)''')
        cur.execute('DROP TABLE IF EXISTS clinic_menus_POR')
        cur.execute('''CREATE TABLE clinic_menus_POR (item_name TEXT, price INTEGER, 
                    clinic_id INTEGER,city_id INTEGER)''')
        cur.execute('DROP TABLE IF EXISTS clinic_menus_DET')
        cur.execute('''CREATE TABLE clinic_menus_DET (item_name TEXT, price INTEGER, 
                    clinic_id INTEGER,city_id INTEGER)''')
        cur.execute('DROP TABLE IF EXISTS clinic_menus_BAL')
        cur.execute('''CREATE TABLE clinic_menus_BAL (item_name TEXT, price INTEGER, 
                    clinic_id INTEGER,city_id INTEGER)''')
        cur.execute('DROP TABLE IF EXISTS clinic_latlng')
        cur.execute('''CREATE TABLE clinic_latlng (lat FLOAT, lng FLOAT, clinic_id INTEGER )''')
        conn.commit()
        cur.close()
        return 
        
    print(city_name)
    if '-mi' in city_name and city_name != 'Detroit-mi':
        cur.execute('SELECT id FROM city_table WHERE name=?', ('Detroit-mi',))
    elif '-md' in city_name and city_name != 'baltimore-md':
        cur.execute('SELECT id FROM city_table WHERE name=?', ('baltimore-md',))
    elif '-nv' in city_name and city_name!='Las Vegas':
        cur.execute('SELECT id FROM city_table WHERE name=?', ('Las Vegas',))
    else:
        cur.execute('SELECT * FROM city_table WHERE name=?', (city_name,))
        entry = cur.fetchone()
        if entry is None:
            cur.execute('INSERT INTO city_table (name) VALUES (?)',(city_name,))
            print('new entry added. total must be 9.')
        cur.execute('SELECT id FROM city_table WHERE name=?',(city_name,))
        
    city_id = cur.fetchone()[0]
    conn.commit()
    cur.close()
    return city_id    
        